In [59]:
import os
import descartes
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

dir = "."
dataDir = os.path.abspath(dir)

In [60]:
#Read CoVerage data lake
df_coverage = pd.read_csv(dataDir + "/TestLake.csv", delimiter=",")
df_coverage.head()

,id (S),timestamp (S),coronaVirus (N),coughing (N),diarrhea (N),gender (N),generalHealth (N),headache (N),insomnia (N),limbPain (N),loneliness (N),numberOfContacts (N),postalCode (S),runnyNose (N),soreThroat (N),temperature (N),userId (S),yearOfBirth (N)
0,002-81-8444,1584808212496,2,2,1,2,2,1,2,1,1,486,82278,3,2,2,Escher,2001
1,003-66-7460,1584808207788,3,1,1,1,5,2,5,2,4,39,31100,2,2,2,Pollock,1971
2,011-71-5874,1584808207109,1,2,2,1,4,2,4,2,4,129,65623,2,1,2,Seurat,1959
3,022-09-5822,1584808219209,3,2,1,2,2,2,5,3,5,208,76623,2,2,1,Rubens,1986
4,037-19-9715,1584808219473,2,2,2,2,3,2,3,2,2,888,51846,1,2,2,Titian,2000


In [64]:
#aggregate df_coverage
#count propable infections (2) 
df_count_guess = df_coverage.copy()
df_count_guess["guessed"] = np.where(df_count_guess["coronaVirus (N)"]==2, 1, 0)
df_count_guess.head(10)

,id (S),timestamp (S),coronaVirus (N),coughing (N),diarrhea (N),gender (N),generalHealth (N),headache (N),insomnia (N),limbPain (N),loneliness (N),numberOfContacts (N),postalCode (S),runnyNose (N),soreThroat (N),temperature (N),userId (S),yearOfBirth (N),guessed
0,002-81-8444,1584808212496,2,2,1,2,2,1,2,1,1,486,82278,3,2,2,Escher,2001,1
1,003-66-7460,1584808207788,3,1,1,1,5,2,5,2,4,39,31100,2,2,2,Pollock,1971,0
2,011-71-5874,1584808207109,1,2,2,1,4,2,4,2,4,129,65623,2,1,2,Seurat,1959,0
3,022-09-5822,1584808219209,3,2,1,2,2,2,5,3,5,208,76623,2,2,1,Rubens,1986,0
4,037-19-9715,1584808219473,2,2,2,2,3,2,3,2,2,888,51846,1,2,2,Titian,2000,1
5,037-65-3087,1584808215891,2,2,2,3,4,1,5,2,2,493,15787,2,1,2,Munch,1957,1
6,044-84-6959,1584808217823,2,1,2,1,3,1,5,1,5,448,94203,2,2,2,Klimt,1970,1
7,046-68-1426,1584808201461,2,1,2,2,2,1,2,2,1,266,11215,1,1,1,Pissarro,1956,1
8,058-15-8475,1584808227834,1,1,2,3,5,2,3,2,5,258,84009,2,3,2,Diego Rivera,1994,0
9,078-75-5417,1584808191187,2,2,2,1,2,2,1,2,2,857,72412,1,1,2,Winslow Homer,1996,1


In [67]:
df_aggregate = df_count_guess.groupby(["postalCode (S)"])["guessed"].sum()
df_aggregate.describe()

count    100.000000
mean       0.500000
std        0.502519
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: guessed, dtype: float64

In [ ]:
#count real infections (3) 
df_count_positive = 

In [ ]:
# Read PLZ polygons
# Make sure you read postal codes as strings, otherwise 
# the postal code 01110 will be parsed as the number 1110. 
plz_shape_df = gpd.read_file('plz-gebiete/plz-gebiete.shp', dtype={'plz': str})
plz_shape_df.head()



In [ ]:
#Init plot parameter
plt.rcParams['figure.figsize'] = [16, 11]

In [ ]:
# Define cities to plot 
top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Cologne': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt am Main': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'Munich': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nuremberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052)
}

In [ ]:
# Merge plz_df and Coverage lake.
plz_shape_df = pd.merge(
    left=plz_shape_df, 
    right=df_coverage, 
    on
    left_on='plz',
    right_on='postalCode',
    how='left'
)

plz_shape_df.head()

In [ ]:
# Define plot feature
plz_shape_df = plz_shape_df \
    .assign(first_dig_plz = lambda x: x['plz'].str.slice(start=4, stop=5))

In [ ]:
#Plot
fig, ax = plt.subplots()



#plot plz shape
plz_shape_df.plot(
    ax=ax, 
    column='first_dig_plz', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'CoVerage Map', 'loc':'lower right'},
    cmap='tab20',
    alpha=0.9
)

#plot cities
for c in top_cities.keys():
    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='CoVerage', 
    aspect=1.3,
    facecolor='white'
);